# **YAI 2021 Fall Project - Medical Project Team**

**Participants**

* Dongha Kim - Yonsei Univ. College of Medicine.

* Donggeon Bae - Yonsei. Univ. Dept. of Electrical and Electronic Engineering.

* Junho Lee - Yonsei Univ. Dept. of Computer Engineering.


## Description

TBD


## Dataset Information

**Cancer Instance Segmentation and Classification: kaggle**

19 Tissue types (Breast, Colon, Bile-duct, Esophagus, Uterus, Lung, Cervix, Head&Neck, Skin, Adrenal Gland, Kidney, Stomach, Prostate, Testis, Liver, Thyroid, Pancreas, Ovary, Bladder). Note, that it also unifies existing datasets within it, we have carefully labelled these under a single nuclei categories schema that is common to all 19 tissues.

* https://www.kaggle.com/andrewmvd/cancer-inst-segmentation-and-classification

* https://www.kaggle.com/andrewmvd/cancer-instance-segmentation-and-classification-2

* https://www.kaggle.com/andrewmvd/cancer-instance-segmentation-and-classification-3


## Model Architecture

**Baseline: DeepLabV3 + Resnet101**

* https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/


## Full Source Code

* github link: TBD

All non-necessary codes are modularized as package. Watch all codes in github repository.

# Runtime Preparation

## Mount Data Drive

In [ ]:
try:
    from google.colab import drive  # NOQA
except ImportError:
    raise RuntimeError("This notebook must be run on colab runtime!")
else:
    drive.mount('/content/drive')  # NOQA
    %cd "/content/drive/Shareddrives/YAI 2021 가을학기 의료 프로젝트팀"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWiCLUxk0W6otTsQUVHKixUYmxr9ZxN0Ns8CDye3653gHvp2_ukhkyA
Mounted at /content/drive
/content/drive/Shareddrives/YAI 2021 가을학기 의료 프로젝트팀


## View Runtime Information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0 or gpu_info.find('not found') >= 0:
    import os
    if 'TPU_NAME' in os.environ:
        mode = 'xla'
        print('TPU Runtime')
    else:
        mode = 'cpu'
        print('Not connected to a GPU')
else:
    mode = 'cuda'
    print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f'Your runtime has {ram_gb:.1f} gigabytes of available RAM\n'
      f'{"Not" if ram_gb < 20 else "You are"} using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM
You are using a high-RAM runtime!


In [ ]:
import sys
import platform
print(f"OS version: \t\t{platform.platform()}\nPython version:\t\t{sys.version.replace(chr(10), str())}")

OS version: 		Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
Python version:		3.7.11 (default, Jul  3 2021, 18:01:19) [GCC 7.5.0]


## Prepare device and library

In [ ]:
# Prepare device

if mode == 'xla':
    try:
        import torch_xla
    except ImportError:
        !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        !python pytorch-xla-env-setup.py
        !rm -rf pytorch-xla-env-setup.py *.whl
        import torch_xla
    import torch
    import torch_xla.core.xla_model as xm
    # Acquires the default Cloud TPU core and moves the model to it
    device = xm.xla_device()

elif mode == 'cuda':
    import torch
    device = torch.device("cuda")
    # loader_kwargs = dict(pin_memory=True)

else:
    import torch
    device = torch.device("cpu")

print(f"Torch version:\t\t{torch.__version__}\nTorch device:\t\t{device}")

Torch version:		1.9.0+cu102
Torch device:		cuda


In [ ]:
# After all installation, import all libraries used.

!pip install torchinfo

import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import IterableDataset, ChainDataset, RandomSampler, DataLoader
from torchvision.datasets import VisionDataset
from torchvision.transforms import ToTensor
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation import deeplabv3_resnet101
import torchinfo
from tqdm.notebook import tqdm
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2


# Dataset Preparation

**Dataset Information**


19 Tissue types (Breast, Colon, Bile-duct, Esophagus, Uterus, Lung, Cervix, Head&Neck, Skin, Adrenal Gland, Kidney, Stomach, Prostate, Testis, Liver, Thyroid, Pancreas, Ovary, Bladder). Note, that it also unifies existing datasets within it, we have carefully labelled these under a single nuclei categories schema that is common to all 19 tissues.

**This particular directory includes training patches of size 256x256 and their masks, this is one of the folds. In total there are more than 7 thousand training patches within all of the three folds.**

The files within each fold directories are:

* `images.npy` - image patches of 256x256

* `masks.npy` an array of 6 channel instance-wise masks (0: Neoplastic cells, 1: Inflammatory, 2: Connective/Soft tissue cells, 3: Dead Cells, 4: Epithelial, 6: Background)

* `types.py`  tissue type that a given path was extracted from.

**The `masks/` directory and it's contents are licensed under a Creative Commons [Attribution-NonCommercial-ShareAlike 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)  license**

## Make Dataset Class

In [ ]:
class NumpyDataset(VisionDataset):  # 메모리 충분하신 분만 쓰세요

    def __init__(
            self,
            image_path,
            mask_path,
            type_path=None,
            transform=ToTensor(),
            target_transform=ToTensor(),
    ):
        VisionDataset.__init__(
            self,
            root=None,  # type: ignore
            transforms=None,
            transform=transform,
            target_transform=target_transform,
        )

        self.image_array = np.load(image_path)
        self.mask_array = np.load(mask_path)
        assert self.image_array.shape[0] == self.mask_array.shape[0]
        if type_path is not None:
            self.type_array = np.load(type_path)
            assert self.image_array.shape[0] == self.type_array.shape[0]
        else:
            self.type_array = None

    def __len__(self):
        return self.image_array.shape[0]

    def __iter__(self):
        for index in range(len(self)):
            image, mask = self.image_array[index], self.mask_array[index]
            if self.transform is not None:
                image = self.transform(image)
            if self.target_transform is not None:
                mask = self.target_transform(mask)
            yield image, mask

    def __getitem__(self, index):
        image, mask = self.image_array[index], self.mask_array[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            mask = self.target_transform(mask)
        return image, mask


class LazyEvalNumpyDataset(IterableDataset, NumpyDataset):

    __length_cache = None

    def __init__(
            self,
            image_path,
            mask_path,
            type_path=None,
            transform=ToTensor(),
            target_transform=ToTensor(),
            sampler_class=None,
            sampler_kwargs=None,
            length=None,
    ):
        VisionDataset.__init__(
            self,
            root=None,  # type: ignore
            transforms=None,
            transform=transform,
            target_transform=target_transform,
        )

        self.image_path = image_path
        assert os.path.isfile(image_path)
        self.mask_path = mask_path
        assert os.path.isfile(mask_path)
        self.type_path = type_path
        assert type_path is None or os.path.isfile(type_path)
        if length is not None:
            self.__length_cache = length

        if sampler_class is not None:
            self.sampler = sampler_class(self, **(sampler_kwargs or {}))
        else:
            self.sampler = None

    def __len__(self):
        if self.__length_cache is None:
            self.__length_cache = np.load(self.image_path).shape[0]
        return self.__length_cache

    def __iter__(self):
        image_array = np.load(self.image_path)
        mask_array = np.load(self.mask_path)
        if self.__length_cache is None:
            self.__length_cache = image_array.shape[0]
        sampler = self.sampler or range(len(image_array))
        for index in sampler:
            image, mask = image_array[index], mask_array[index]
            if self.transform is not None:
                image = self.transform(image)
            if self.target_transform is not None:
                mask = self.target_transform(mask)
            yield image, mask

    def __getitem__(self, item):
        raise NotImplementedError


## Instantiate Dataset

In [ ]:
batch_length = [886, 885, 885, 841, 841, 841, 908, 907, 907]

train_subset = [0, 2, 3, 4, 6, 8]
val_subset = [1, 5]
test_subset = [7]

path_format = os.path.join("data", "processed", "{0}", "{1}.npy")
train_dataset = ChainDataset([
    LazyEvalNumpyDataset(
        image_path=path_format.format("images", i),
        mask_path=path_format.format("masks", i),
        length=batch_length[i],
        sampler_class=RandomSampler,  # issue
    ) for i in train_subset
])
val_dataset = ChainDataset([
    LazyEvalNumpyDataset(
        image_path=path_format.format("images", i),
        mask_path=path_format.format("masks", i),
        length=batch_length[i],
    ) for i in val_subset
])
test_dataset = ChainDataset([
    LazyEvalNumpyDataset(
        image_path=path_format.format("images", i),
        mask_path=path_format.format("masks", i),
        length=batch_length[i],
    ) for i in test_subset
])

# Network Preparation

## Segmentation Network

**Baseline Model**: DeepLabV3-Resnet101

Other Architectures

* A-HRNet: https://ieeexplore.ieee.org/document/9253142
* See Also: https://arxiv.org/pdf/2005.10821.pdf

In [ ]:
# # Pretrained Model
net = deeplabv3_resnet101(pretrained=True, progress=False)
net.classifier = DeepLabHead(2048, 6)
# net.aux_classifier = nn.Sequential()
net.aux_classifier = FCNHead(1024, 6)
# # Non-pretrained Model
# net = deeplabv3_resnet101(pretrained=False, num_classes=6)

trainable_backbone_layers = ['layer4']
for n, p in net.named_parameters():
    if n.startswith('backbone') and n.split('.')[1] not in trainable_backbone_layers:
        p.requires_grad = False

NUM_GPUS = torch.cuda.device_count()
if NUM_GPUS > 1:
    net = torch.nn.DataParallel(net)

net.to(device)

torchinfo.summary(net, (1, 3, 256, 256))

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                             Output Shape              Param #
DeepLabV3                                          --                        --
├─DeepLabHead: 1                                   --                        --
│    └─ASPP: 2                                     --                        --
│    │    └─ModuleList: 3-1                        --                        15,206,912
├─IntermediateLayerGetter: 1-1                     [1, 2048, 32, 32]         --
│    └─Conv2d: 2-1                                 [1, 64, 128, 128]         9,408
│    └─BatchNorm2d: 2-2                            [1, 64, 128, 128]         128
│    └─ReLU: 2-3                                   [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                              [1, 64, 64, 64]           --
│    └─Sequential: 2-5                             [1, 256, 64, 64]          --
│    │    └─Bottleneck: 3-2                        [1, 256, 64, 64]          75,008
│    │    └─Bottlen

## Loss Network

* More Multi-Label Segmentation Losses: https://jeune-research.tistory.com/entry/Loss-Functions-for-Image-Segmentation-Region-Based-Losses

* See also: https://smp.readthedocs.io/en/latest/losses.html



In [ ]:
class DiceLoss(torch.nn.Module):  # Multi label segmentation loss

    epsilon = 1e-7

    def forward(self, logit, target):
        pred = logit.softmax(dim=-3)
        intersection = (pred * target).sum(dim=(-1, -2)) + self.epsilon
        union = (pred + target).sum(dim=(-1, -2)) + self.epsilon * 2
        loss = 1 - (2.0 * intersection / union)
        return loss.mean()


# Training

## Define Trainer Function

In [ ]:
def train_one_epoch(model, criterion, optimizer, scheduler, train_loader, val_loader, device, epoch):

    model.train()

    if epoch == 0:  # warm_up_scheduler
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(train_loader) - 1)

        def f(x):
            if x >= warmup_iters:
                return 1
            alpha = float(x) / warmup_iters
            return warmup_factor * (1 - alpha) + alpha

        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, f)

    loss_f = None
    inner_tq = tqdm(train_loader, total=len(train_loader), leave=False, desc=f'Iteration {epoch} train')

    for images, masks in inner_tq:

        images = images.to(device).float()
        masks = masks.clamp(0., 1.).to(device).float()  # IMPORTANT: clamp

        prediction = model(images)
        # prediction = prediction.softmax(dim=-3)
        loss = criterion(prediction, masks)
        loss.backward()

        if torch.cuda.device_count() > 1:
            loss = loss.mean()  # mean() to average on multi-gpu.

        optimizer.step()
        optimizer.zero_grad()

        if epoch == 0:
            scheduler.step()

        if loss_f is not None:
            loss_f = 0.98 * loss_f + 0.02 * loss.item()
        else:
            loss_f = loss.item()
        inner_tq.set_postfix(loss=loss_f)

    if epoch != 0 and scheduler is not None:
        scheduler.step()

    print(f"\rIteration {epoch} train loss: {loss_f:.4f}")

    if val_loader is not None:
        model.eval()
        inner_tq = tqdm(val_loader, total=len(val_loader), leave=False, desc=f'Iteration {epoch} eval')
        with torch.no_grad():
            loss_f = count = 0
            for images, masks in inner_tq:
                images = images.to(device).float()
                masks = masks.clamp(0., 1.).to(device).float()  # IMPORTANT: clamp
                prediction = model(images)
                loss_f += criterion(prediction, masks).item()
                count += 1
            loss_f = loss_f / count if count else None
            inner_tq.set_postfix(loss=loss_f)
            print(f"Iteration {epoch} eval loss: {loss_f:.4f}")


## Train and Evaluate

In [ ]:
# Lazy-eval iterable dataset: do not set sampler or shuffle options
num_epoch = 10

batch_size = 100
num_workers = 1

train_loader = DataLoader(train_dataset, batch_size, num_workers=num_workers, drop_last=False)
val_loader = DataLoader(val_dataset, batch_size, num_workers=num_workers, drop_last=False)

loss_func = DiceLoss()
optimizer = optim.SGD(
    [p for p in net.parameters() if p.requires_grad],
    lr=1e-3, momentum=0.9, weight_decay=0.0005
)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


def load_state_dict(d):
    net.load_state_dict(d['model'])
    optimizer.load_state_dict(d['optimizer'])
    lr_scheduler.load_state_dict(d['lr_scheduler'])


def state_dict():
    from collections import OrderedDict
    d = OrderedDict()
    d['model'] = net.state_dict()
    d['optimizer'] = optimizer.state_dict()
    d['lr_scheduler'] = lr_scheduler.state_dict()
    return d


In [ ]:
os.makedirs('checkpoint', exist_ok=True)
for ep in range(num_epoch):
    train_one_epoch(net, loss_func, optimizer, lr_scheduler, train_loader, val_loader, device, ep)
    torch.save(net.state_dict(), os.path.join('checkpoint', 'baseline_epoch{}.pt').format(ep))

## Test

In [ ]:
net.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [ ]:
torch.save(state_dict(), 'baseline_final.pth')